## Распознование текста из микрофона

In [ ]:
sudo apt-get install portaudio19-dev

In [1]:
!environment/bin/pip install SpeechRecognition
!environment/bin/pip install PyAudio

In [2]:
import speech_recognition as sr

In [3]:
recognizer = sr.Recognizer()

In [4]:
recognizer.listen??

In [4]:
with sr.Microphone() as micro:
    print(1)
    audio = recognizer.listen(micro)
    print(2)

1
2


In [54]:
with open('data/microphone-results.wav', "wb") as f:
    f.write(audio.get_wav_data())

In [5]:
recognizer.recognize_google(audio, language='en-EN')

'speak speak speak speak speak speak'

## Распознование текста из аудиофайла

In [31]:
recognizer = sr.Recognizer()

In [58]:
with sr.AudioFile('data/microphone-results.wav') as source:
    audio1 = recognizer.record(source)

In [59]:
recognizer.recognize_google(audio1, language='en-EN')

'this is a sound detection chick 111 detection check'

## Text topic extraction

In [36]:
!py3_env/bin/pip install spacy
!py3_env/bin/pip install pymorphy2

In [37]:
import spacy

In [39]:
# spacy.load('ru')

In [60]:
from spacy.lang.en import English

In [61]:
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [63]:
text = recognizer.recognize_google(audio, language='en-EN')
tokenize(text)

['this', 'is', 'a', 'sound', 'detection', 'chick', '111', 'detection', 'check']

In [3]:
!environment/bin/pip install nltk

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [16]:
get_lemma("их")

'их'

In [78]:
for token in tokenize(text):
    print(get_lemma(token), get_lemma2(token))

this this
be is
a a
sound sound
detection detection
chick chick
111 111
detection detection
check check


In [11]:
nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
list(en_stop)[:5]

['со', 'при', 'чтобы', 'может', 'без']

In [90]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return [get_lemma(token) for token in tokens 
            if len(token) > 4 and token not in en_stop]

In [91]:
text_data = [prepare_text_for_lda(text)]
text_data

[['sound', 'detection', 'chick', 'detection', 'check']]

In [82]:
!py3_env/bin/pip install gensim

In [86]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in text_data]
print(corpus)

Dictionary(4 unique tokens: ['check', 'chick', 'detection', 'sound'])
[[(0, 1), (1, 1), (2, 2), (3, 1)]]


In [89]:
dictionary.doc2bow?? # document to bag of words

In [88]:
import gensim

NUM_TOPICS = 2
NUM_WORDS = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.251*"detection" + 0.250*"check" + 0.250*"sound"')
(1, '0.359*"detection" + 0.214*"chick" + 0.214*"sound"')


## загрузка аудиофайла из youtube

In [31]:
!py3_env/bin/pip install youtube_dl
!py3_env/bin/pip install oauth2client
!py3_env/bin/pip install pydub

In [14]:
import youtube_dl

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'outtmpl': '1',
    'prefer_ffmpeg': True,
    'keepvideo': False
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=J5O5iRpLXKY&t=637s'])

[youtube] J5O5iRpLXKY: Downloading webpage
[youtube] J5O5iRpLXKY: Downloading video info webpage
[download] Destination: 1
[download] 100% of 15.11MiB in 00:13.00KiB/s ETA 00:000
[ffmpeg] Destination: wav
Deleting original file 1 (pass -k to keep)


In [15]:
from pydub import AudioSegment
t1 = 110 * 1000 #Works in milliseconds
t2 = 200 * 1000
newAudio = AudioSegment.from_wav("data/1.wav")
newAudio = newAudio[t1:t2]
newAudio.export('data/3.wav', format="wav")

<_io.BufferedRandom name='data/3.wav'>

In [57]:
recognizer = sr.Recognizer()
with sr.AudioFile('data/3.wav') as source:
    audio1 = recognizer.record(source)
recognizer.recognize_google(audio1, language='ru_RU')

'чтобы после очень крутой 18 года попробовать себя на западе А это произошло наверное летом летом 18 летом 18 года Так как только артист увидел офсет увидел эту ссылку Он резко отправил своим продюсером и сказал у меня переписку показывали потом эти же продюсер искал это моё секретное оружие Давайте звать парня и потихоньку очень медленно начинался запускается процесс То есть как бы это ни было моментально там о Попов и мы уже Я уже тут В начале обсуждал Ну поскольку у них стартовал тур совместно с дрейком не улавливает of цитата 11 чуваков из группы migos когда архитектуре это всегда любой клип запись альбома идёт очень медленно потому что они ежедневно просто путешествую фотосессии концерты в каждом городе по два по три концерта все солдаут были и наш тоже клип он сильно двигался медленно но ближе наверное в октябре уже нам Мне позвонили сказали всё Поехали Давай всё снимать придумывай идею У артиста из Кто такие заметную столько-то заметок что он хочет видеть и мы сейчас я с этим св

## Analyse podcast topics with wor2vec

In [4]:
import re
import nltk
import numpy as np
from gensim.models import Word2Vec, ldamodel
from collections import Counter

NUM_TOPICS = 10
NUM_WORDS = 5

In [2]:
with open('data/podcast.txt', 'r+') as f:
    text = f.read()
text[100:200]

'й как нервничал тогда Секс это правда что вы гений с Юрий будет дуть дуть будет Юрий очень стрёмно в'

In [5]:
[m.start(0) for m in re.finditer(' [А-Я][а-я]', text[:50])], text[:50]

([4, 14], 'дудь Агент ФСБ Если бы гебельс был бы жив он застр')

In [6]:
sentences = []

sentences_starts = [m.start(0) for m in re.finditer(' [А-Я][а-я]', text)]

if sentences_starts[0] != 0:
    sentences_starts = [0] + sentences_starts

sentences_starts += [len(sentences_starts)]

for i in range(len(sentences_starts) - 1):
    start_index = sentences_starts[i]
    end_index = sentences_starts[i+1] - 1
    sentence = text[start_index: end_index+1].lower()
    sentences.append(sentence)

In [7]:
print(len(sentences))
print(Counter([len(sentence) for sentence in sentences]))

548
Counter({10: 23, 8: 22, 11: 21, 7: 21, 15: 14, 13: 12, 17: 11, 27: 10, 24: 10, 9: 10, 16: 9, 52: 9, 19: 8, 18: 8, 25: 8, 73: 7, 23: 7, 4: 6, 31: 6, 14: 6, 12: 6, 35: 6, 21: 5, 5: 5, 47: 5, 36: 5, 6: 5, 33: 5, 26: 5, 22: 5, 39: 5, 42: 5, 80: 4, 49: 4, 75: 4, 59: 4, 66: 4, 43: 4, 45: 4, 62: 4, 51: 4, 20: 4, 44: 4, 96: 3, 38: 3, 106: 3, 104: 3, 95: 3, 110: 3, 81: 3, 29: 3, 34: 3, 126: 3, 76: 3, 91: 3, 64: 3, 48: 3, 46: 3, 68: 3, 30: 3, 90: 3, 37: 3, 70: 3, 86: 3, 56: 3, 108: 3, 130: 3, 107: 2, 122: 2, 28: 2, 160: 2, 53: 2, 40: 2, 55: 2, 268: 2, 133: 2, 124: 2, 57: 2, 60: 2, 58: 2, 65: 2, 120: 2, 67: 2, 88: 2, 87: 2, 116: 2, 275: 2, 61: 2, 209: 2, 156: 2, 138: 2, 69: 2, 109: 2, 176: 2, 166: 2, 54: 2, 72: 2, 113: 2, 50: 2, 32: 2, 153: 2, 100: 2, 350: 1, 142: 1, 152: 1, 101: 1, 92: 1, 346: 1, 286: 1, 255: 1, 216: 1, 256: 1, 530: 1, 287: 1, 99: 1, 478: 1, 318: 1, 167: 1, 3: 1, 188: 1, 165: 1, 170: 1, 249: 1, 164: 1, 322: 1, 147: 1, 280: 1, 134: 1, 333: 1, 302: 1, 82: 1, 94: 1, 155: 1, 191

In [8]:
tokenized_sentences = []
en_stop = set(nltk.corpus.stopwords.words('russian'))
tokens_before, tokens_after = 0, 0

for sentence in sentences:
    tokens = sentence.split()
    tokens_before += len(tokens)
    filtered_tokens = [token for token in tokens if token not in en_stop]
    tokens_after += len(filtered_tokens)
    tokenized_sentences.append(filtered_tokens)

print(tokens_before, tokens_after)
print(Counter([len(sentence) for sentence in tokenized_sentences]))

5657 3405
Counter({1: 139, 2: 77, 3: 51, 4: 35, 5: 32, 6: 31, 7: 27, 9: 19, 11: 16, 8: 14, 10: 12, 12: 11, 14: 9, 13: 9, 15: 8, 22: 8, 0: 7, 19: 6, 17: 5, 16: 5, 34: 3, 24: 3, 21: 3, 26: 2, 25: 2, 18: 2, 31: 2, 28: 2, 33: 1, 45: 1, 50: 1, 29: 1, 27: 1, 35: 1, 23: 1, 20: 1})


In [9]:
model = Word2Vec(tokenized_sentences, size=150, window=10, min_count=1, workers=4)
model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=10)

(31810, 34050)

In [10]:
model.wv.vocab

{'дудь': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d79e8>,
 'агент': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7b00>,
 'фсб': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7ba8>,
 'гебельс': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7ac8>,
 'жив': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7c88>,
 'застрелился': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7cf8>,
 'ещё': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7cc0>,
 'тошнит': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7c18>,
 'нервничай': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7a20>,
 'нервничал': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7b70>,
 'секс': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7b38>,
 'это': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7d68>,
 'правда': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7e48>,
 'гений': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7da0>,
 'юрий': <gensim.models.keyedvectors.Vocab at 0x7f93ba3d7e10>,
 'дуть': <gensim.models.keyedvect

In [11]:
word_vectors = []
tokens = []

for sentence in tokenized_sentences:
    tokens += sentence
    word_vectors += [model.wv[token] for token in sentence]# if token in model.wv.vocab]

word_vectors[:5]

[array([-1.3315046e-03, -2.6911697e-03,  1.1014979e-03,  3.2491372e-03,
         2.0084872e-03,  3.4838115e-04,  4.2996780e-04,  1.8123223e-03,
        -1.6897842e-03,  3.6773330e-03, -1.2886504e-03,  1.8777865e-03,
         1.2847335e-03,  2.4661485e-03, -8.6376996e-05, -2.3547602e-03,
        -2.7288895e-03,  2.9353539e-03, -1.5372210e-03,  1.0281428e-03,
        -2.5492883e-04,  1.0745566e-03,  1.2939832e-03,  3.3880118e-03,
        -1.6419158e-03,  2.5321371e-03,  2.6287101e-04,  1.1206865e-03,
        -1.8359745e-03,  1.9854086e-03, -1.3059637e-03, -3.3479012e-03,
        -4.7493685e-04,  7.6750416e-04, -1.6661464e-03,  1.9404824e-03,
         2.8505994e-03,  2.6663020e-03, -2.2956342e-03, -1.8350807e-03,
         1.3631601e-04,  1.1481865e-03,  2.1398952e-03,  3.1430118e-03,
        -6.1570382e-04,  2.0299712e-03, -2.5762955e-03,  6.8823947e-04,
         1.9170044e-03,  3.7741996e-04,  2.5561983e-03, -8.1315498e-05,
         1.9260051e-03, -2.8062998e-03, -8.0923637e-05,  1.62886

In [79]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=NUM_TOPICS, random_state=0).fit(word_vectors)

In [80]:
from collections import Counter
clusters_counter = Counter(kmeans.predict(word_vectors))
clusters_counter

Counter({4: 568,
         5: 665,
         8: 122,
         7: 365,
         9: 431,
         0: 462,
         3: 429,
         1: 118,
         2: 174,
         6: 71})

In [83]:
for topic_label in sorted(clusters_counter.keys(), 
                          key=lambda cluster_label:clusters_counter[cluster_label], reverse=True):
    topic_tokens = np.array(tokens)[kmeans.labels_ == topic_label]
    topic_counter = Counter(topic_tokens)
    sorted_tokens = sorted(topic_counter.keys(), key=lambda token:topic_counter[token], reverse=True)
    print(topic_label, sorted_tokens[:5])

5 ['крыма', 'дуть', 'юра', 'интернете', 'владимир']
4 ['юрий', 'владимира', 'россию', 'войну', 'украинский']
0 ['жизнь', 'путин', 'кгб', 'назад', 'блин']
9 ['россии', 'украины', 'чемпионат', 'вопрос', 'просто']
3 ['россия', 'российской', 'говорят', 'невозможно', '1']
7 ['крым', 'украине', 'украина', 'человека', 'происходит']
2 ['который', 'почему', 'мира', 'говорю', 'хочу']
8 ['которые', 'люди', 'сказал', 'фсб', 'президентом']
1 ['это']
6 ['всё', 'очень', 'человек']


In [47]:
for i, cluster in enumerate(sorted(clusters_counter, 
                                   key=lambda cluster: clusters_counter[cluster], reverse=True)):
    cluster_vector = kmeans.cluster_centers_[cluster]
    topic = model.wv.similar_by_vector(cluster_vector, topn=NUM_WORDS, restrict_vocab=None)
    print('TOPIC_' + str(i+1), *[word[0] for word in topic])

TOPIC_1 человек всё людей которые говорю
TOPIC_2 это хочу сегодня очень думаю
TOPIC_3 это очень которые люди человек
TOPIC_4 это очень человек всё которые
TOPIC_5 это всё человек очень фамилии
TOPIC_6 это очень всё человек фамилии
TOPIC_7 это человек фамилии очень который
TOPIC_8 которые это люди очень человек
TOPIC_9 это всё очень человек фамилии
TOPIC_10 всё очень человек это которые


In [16]:
from gensim import corpora

dictionary = corpora.Dictionary([tokens])
# print(dictionary)
corpus = [dictionary.doc2bow(text) for text in [tokens]]
# print(corpus)


lda_model = ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = lda_model.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.001*"я" + 0.001*"не" + 0.001*"и" + 0.001*"это" + 0.001*"в"')
(1, '0.028*"и" + 0.027*"я" + 0.027*"не" + 0.026*"в" + 0.022*"что"')
(2, '0.001*"и" + 0.001*"не" + 0.001*"я" + 0.001*"в" + 0.001*"это"')
(3, '0.001*"я" + 0.001*"что" + 0.001*"и" + 0.001*"в" + 0.001*"не"')
(4, '0.001*"и" + 0.001*"не" + 0.001*"в" + 0.001*"я" + 0.001*"что"')
(5, '0.001*"и" + 0.001*"не" + 0.001*"я" + 0.001*"что" + 0.001*"в"')
(6, '0.001*"я" + 0.001*"не" + 0.001*"в" + 0.001*"что" + 0.001*"и"')
(7, '0.001*"не" + 0.001*"и" + 0.001*"я" + 0.001*"в" + 0.001*"что"')
(8, '0.001*"не" + 0.001*"в" + 0.001*"и" + 0.001*"я" + 0.001*"что"')
(9, '0.001*"и" + 0.001*"я" + 0.001*"это" + 0.001*"в" + 0.001*"что"')


## Draw words vectors

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [16]:
from sklearn.manifold import TSNE

In [29]:
word_vectors_2d = TSNE(n_components=2).fit_transform(word_vectors)
draw_vectors(word_vectors_2d[:, 0], word_vectors_2d[:, 1], token=tokens)

Figure(id='1519', ...)

In [30]:
uniq_tokens = np.unique(tokens)
uniq_word_vectors = [model.wv[token] for token in uniq_tokens]

In [31]:
uniq_word_vectors_2d = TSNE(n_components=2).fit_transform(uniq_word_vectors)
draw_vectors(uniq_word_vectors_2d[:, 0], uniq_word_vectors_2d[:, 1], token=uniq_tokens)

Figure(id='1662', ...)

In [35]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=NUM_TOPICS, random_state=0).fit(uniq_word_vectors)

In [36]:
from collections import Counter
clusters_counter = Counter(kmeans.predict(word_vectors))
clusters_counter

Counter({6: 415,
         2: 319,
         4: 434,
         7: 290,
         5: 225,
         0: 576,
         9: 295,
         1: 261,
         8: 288,
         3: 302})

In [38]:
for topic_label in sorted(clusters_counter.keys(), 
                          key=lambda cluster_label:clusters_counter[cluster_label], reverse=True):
    topic_tokens = np.array(tokens)[kmeans.predict(word_vectors) == topic_label]
    topic_counter = Counter(topic_tokens)
    sorted_tokens = sorted(topic_counter.keys(), key=lambda token:topic_counter[token], reverse=True)
    print(topic_label, sorted_tokens[:5])

0 ['россии', 'мира', 'россия', 'украине', 'чемпионат']
4 ['это', 'всё', 'которые', 'очень', 'который']
6 ['украины', 'крым', 'российской', 'крыма', '1']
2 ['войну', 'владимир', 'понимают', 'захватом', 'министра']
3 ['наш', 'момент', 'любой', 'сама', 'хотя']
9 ['владимира', 'интернете', 'взгляд', 'кучма', 'гражданин']
7 ['говорят', 'других', 'второй', 'крыму', '94']
8 ['юрий', 'давайте', 'первый', 'спецслужб', 'называть']
1 ['дуть', 'россию', 'юра', 'важно', 'владимирович']
5 ['янукович', 'украинский', 'путина', 'назад', 'мало']
